# Neural Newtworks with Sequential and Functional API

Ignore information messsgaes from Tensorflow

In [23]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Import Tensorflow and other libraries
Tensorflow is an open source library for numerical computation, specializing in machine learning applications. Its flexible architecture allows easy deployment of computation across a variety of platforms (CPUs, GPUs, TPUs), and from desktops to clusters of servers to mobile and edge devices. 

### Keras
Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation. It is used to buid and train deep learning models.

### Sequential model
Sequential model is a linear stack of layers. It can be created by passing a list of layer instances to the constructor. Or the layers can be added one by one via the add() method.

In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

## Load the dataset
The dataset used in this project is the MNIST dataset. It is a dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. The labels are numbers between 0 and 9 indicating which digit they represent.

In [25]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Print the shape of the dataset, the shape of the labels and the number of classes.
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}, classes: {(set(y_train))}")

x_train shape: (60000, 28, 28), y_train shape: (60000,), classes: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


- **Flatten** the dataset so that it can be used in the model. The flatten layer transforms the format of the images from a 2d-array (of 28 by 28 pixels), to a 1d-array of 28 * 28 = 784 pixels.
- We then convert it to float32 (less computation heavy) and **normalize** it to values between 0 and 1 (faster training). Instead of 255, the maximum value of a pixel color channel, we use 255.0 to ensure that we are performing floating point division. This minimizes computation and ensures that our gradients are in a reasonable range.

In [26]:
# Using NumPy. Float 64 is unnecessary computationally.
x_train = x_train.reshape(-1, 28*28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0

# Equivalent can be done using tensorflow
# x_train = tf.convert_to_tensor(x_train) / 255.0

## Sequential API
Very convenient, but not very flexible. It is easy to build models layer by layer, but it is not easy to build models that share layers or have multiple inputs or outputs. Hence, this is good for **one-input to one-output** stacks of layers, but not for more complex ones.

In [27]:
# Last layer does not have activation function as it is the output layer
model = keras.Sequential(
    [
        keras.Input(shape=(28*28)),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu', name='my_layer'),
        layers.Dense(10),
    ]
)

# Print model summary: General Debugging Tool
# In order to print the summary before we send the data to the model, we need to specify the input shape
# hence we use keras.Input(shape=(28*28)) in the model definition.
model.summary()

# Another way to define the model
# model = keras.Sequential()
# model.add(keras.Input(shape=(784)))
# model.add(layers.Dense(512, activation='relu'))
""" If you need to debug in the middle of the model, you can print model.summary() and then add the next layer."""
# model.summary()
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(10))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
my_layer (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


' If you need to debug in the middle of the model, you can print model.summary() and then add the next layer.'

### Extracting specific layer features
Extracting specific layer features from a neural network can be a valuable technique for debugging and understanding the behavior of the original model. Here's how it can aid in the debugging process:

1. **Interpreting Model Behavior**:
   - Neural networks, especially deep ones, can be complex and challenging to interpret. By extracting features from specific layers, you can gain insights into what the model is learning at various stages of processing.
   - For example, you might want to **inspect** the **activations** of certain intermediate layers to understand whether they are responding to specific patterns, edges, textures, or higher-level features in the input data.

2. **Visualization**:
   - Extracted layer features can be visualized to provide a better understanding of what the model is doing. Visualization techniques, such as heatmaps or activation maps, can help highlight regions in the input data that are particularly important for a given layer.

3. **Identifying Problems**:
   - By inspecting the features extracted from intermediate layers, you can detect problems in the model's learning process. For example:
     - If certain layers consistently produce very low activations, it may indicate **vanishing gradients**, and the layer might need adjustments.
     - If activations are extremely high or NaN (not-a-number), it could suggest **exploding gradients** or other numerical stability issues.
     - If the features don't align with your expectations or domain knowledge, it might indicate that the model isn't learning as intended, and you may need to adjust the architecture, data preprocessing, or training process.

4. **Feature Extraction for Debugging**:
   - You can use extracted features for debugging by comparing them to expected features or known patterns in your data.
   - For example, if you're training a model to recognize objects in images, you can extract features from intermediate layers and compare them to what you expect to see. If the features don't match your expectations, you can investigate why the **model is deviating from the desired behavior.**

5. **Transfer Learning and Fine-Tuning**:
   - Extracted features from intermediate layers can also be used for transfer learning or fine-tuning. You can take these features and train a new model on top of them for a specific task or domain, leveraging the knowledge learned by the original model.

The following code is extracting features from the second-to-last layer of the original model for the input data.  It's a common technique in transfer learning or feature extraction where you might want to use pre-trained layers of a model to extract useful features from your data and then use those features for further analysis or as input to another model. 


In [ ]:
# WE create a new Keras model (model) by taking the inputs from an existing model (model.inputs) and 
# setting the outputs to be the output of the second-to-last layer of the original model (model.layers[-2].output).
model = keras.Model(inputs=model.inputs, outputs=[model.layers[-2].output])
# model = keras.Model(inputs=model.inputs, outputs=[model.get_layer('my_layer').output])

# We then train this new model on the same data as the original model input. Then compute
# the output of the second-to-last layer of the original model for the given input data.
feature = model.predict(x_train)

print(f"feature shape: {feature.shape}")
# for feature in features:
#     print(f"feature shape: {feature.shape}")

## Functional API
The functional API is a way to create models that are more flexible than the **Sequential** API. The functional API can handle models with **non-linear topology**, **shared layers**, and even **multiple inputs or outputs**. The main idea that a deep learning model is usually a directed acyclic graph (DAG) of layers. So the functional API is a way to build **graphs of layers**.

In [ ]:
inputs = keras.Input(shape=(784))
x = layers.Dense(512, activation='relu', name='first_layer')(inputs)
x = layers.Dense(256, activation='relu', name='second_layer')(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

### Compile the model
This is where we specify the network configuration.
- **Loss function** — This measures how accurate the model is during training. We want to minimize this function to "steer" the model in the right direction.
- **Optimizer** — This is how the model is updated based on the data it sees and its loss function, setting the learning rate.
    - **Stochastic gradient descent (SGD)** — This is the most basic and easy to understand optimization algorithm. It is usually a good choice to use SGD for shallow networks (e.g. a single-layer perceptron or linear regression network), but it is not recommended for deeper networks.
    - **Adagrad** — This optimizer is well suited for dealing with <ins>sparse data.</ins>
    - **Adadelta** — It is an extension of Adagrad that seeks to reduce its aggressive, monotonically decreasing learning rate. Instead of accumulating all past squared gradients, Adadelta restricts the window of accumulated past gradients to some fixed size w.
    - **RMSProp** — This optimizer is usually a good choice for <ins>recurrent neural networks (RNNs)</ins>.
    - **Adam** — This is a very effective optimizer for deep networks. It combines some of the best properties of both AdaGrad and RMSProp. In addition to storing an exponentially decaying average of past squared gradients like AdaGrad, and an exponentially decaying average of past gradients like RMSProp, Adam also keeps an exponentially decaying average of past gradients squared, similar to momentum.
    - **Adamax** — It is a variant of Adam based on the infinity norm.
    - **Nadam** — It is Adam RMSprop with Nesterov momentum.
    - **Ftrl** — It is an optimizer that implements the FTRL algorithm.
    - **Gradient Descent with Momentum** — This is a standard gradient descent algorithm with momentum. In addition to storing the average of the gradients vt like momentum, it also stores the average of the squared gradients vt like AdaGrad.
- **Metrics** — Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.

We are using `SparseCategoricalCrossentropy` because the labels are integers, not one-hot encoded. If we used `CategoricalCrossentropy`, we would need to one-hot encode the labels. We do one-hot encoding when we have descriptive labels, like "cat", "dog", "bird", etc. Here, we have numeric labels, so we don't need to one-hot encode them.

`from_logits=True`: The <ins>output of the model is not normalized</ins>, hence we need to use from_logits=True
If you use Softmax activation function in the last layer, then you do `from_logits=False` (default).

In [32]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True),
    # optimizer = keras.optimizers.Adam(lr=0.001),
    metrics = ["accuracy"],
)

### Fit and evaluate the model
- **Epochs** — This is the number of times the model is exposed to the training set. It is one complete forward and backward pass of all the training examples. In other words, during one epoch, the model sees and learns from every example in the training dataset once.
- **Batch size** — Batch size refers to the number of training examples utilized in one forward and backward pass (one update of the model's weights) during each iteration in an epoch. Instead of processing the entire dataset at once, it is divided into smaller batches, and the model updates its weights based on the average gradient computed from these batches.
- **Verbose** — This parameter controls how much information is outputted during training.

Epochs and Batch size are hyperparameters that we can tune to improve the model's performance.

Evaluation is done on the test set. The model is not trained on the test set, so it is a good way to check that the model is not overfitting to the training data. The test data is only used once, to evaluate the final model. Hence Epoch is just set to 1. 

In [33]:
print("Training the model...")
model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)

# Pretty print the results
print("\nEvaluating the model:")
results = model.evaluate(x_test, y_test, batch_size=32, verbose=0)
print(f"Test loss: {results[0]:.4f}")
print(f"Test accuracy: {results[1]:.4f}")


Training the model...
Epoch 1/10
1875/1875 - 2s - loss: 0.0052 - accuracy: 0.9985
Epoch 2/10
1875/1875 - 2s - loss: 0.0023 - accuracy: 0.9994
Epoch 3/10
1875/1875 - 2s - loss: 0.0014 - accuracy: 0.9996
Epoch 4/10
1875/1875 - 2s - loss: 0.0011 - accuracy: 0.9999
Epoch 5/10
1875/1875 - 2s - loss: 8.7134e-04 - accuracy: 0.9999
Epoch 6/10
1875/1875 - 2s - loss: 7.3059e-04 - accuracy: 0.9999
Epoch 7/10
1875/1875 - 2s - loss: 6.3113e-04 - accuracy: 1.0000
Epoch 8/10
1875/1875 - 2s - loss: 5.6227e-04 - accuracy: 1.0000
Epoch 9/10
1875/1875 - 2s - loss: 5.1146e-04 - accuracy: 1.0000
Epoch 10/10
1875/1875 - 2s - loss: 4.7308e-04 - accuracy: 1.0000

Evaluating the model:
Test loss: 0.0861
Test accuracy: 0.9852
